In [1]:
import numpy as np
import openpyxl
from pandas import DataFrame, read_csv, merge
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
loan_df = read_csv("loan.csv")

loan_df["loan_type"] = [val.lower().strip() if isinstance(val, str) else val 
                        for val in loan_df.loan_type ]

In [4]:
# get approved loans

appr = loan_df.loc[loan_df.status == 'approved']
appr.index = range(appr.shape[0])

# get Liberian dollar loans

ld_loan = DataFrame( appr.loc[~appr.loan_amt_ld.isnull()] )
ld_loan["loan_amt_ld"] = [float(num) for num in ld_loan["loan_amt_ld"]]
ld_loan.index = range(ld_loan.shape[0])

# get US dollar loans

usd_loan = DataFrame( appr.loc[~appr.loan_amt_usd.isnull()] )
usd_loan["loan_amt_usd"] = [float(num) for num in usd_loan["loan_amt_usd"]]
usd_loan.index = range(usd_loan.shape[0])

In [5]:
# get stats by loan_type

ld_ln_grp = ld_loan.groupby("loan_type", as_index=False).agg({"loan_id": "count", 
                                                              "loan_amt_ld": "sum"})
ld_ln_grp = ld_ln_grp.rename(columns = {"loan_id": "num_ld_loans"})

usd_ln_grp = usd_loan.groupby("loan_type", as_index=False).agg({"loan_id": "count", 
                                                               "loan_amt_usd": "sum"})
usd_ln_grp = usd_ln_grp.rename(columns = {"loan_id": "num_usd_loans"})

# merge groups

merged_ln_grps = merge(ld_ln_grp, usd_ln_grp, on="loan_type", how="outer")

# add total row

row = merged_ln_grps.values.sum(axis=0)
row[0] = "total"

merged_ln_grps.loc[-1] = row
merged_ln_grps.index = range(merged_ln_grps.shape[0])

In [6]:
merged_ln_grps

,loan_type,num_ld_loans,loan_amt_ld,num_usd_loans,loan_amt_usd
0,business,7,138000.0,7,1200.0
1,personal,2,17250.0,8,1000.0
2,total,9,155250.0,15,2200.0


In [7]:
total_cap = 5000
x_rate = 115
total_ld_loan = merged_ln_grps.loc[(merged_ln_grps.loan_type=='total')].loan_amt_ld.values[0]
total_usd_loan = merged_ln_grps.loc[(merged_ln_grps.loan_type=='total')].loan_amt_usd.values[0]

bal_cap = total_cap - ( (total_ld_loan / x_rate) + total_usd_loan)

bal_cap

1450.0

In [8]:
merged_ln_grps.to_csv("weekly_loan_summary.csv", header=True, index=False)

In [9]:
merged_ln_grps.to_excel("weekly_loan_summary.xlsx", index=False)

In [9]:
import smtplib
# import email.mime as mime

from email.mime.message import MIMEMessage, message
# Import the email modules we'll need
from email.mime.text import MIMEText


In [10]:
# Import smtplib for the actual sending function
import smtplib

# Import the email modules we'll need
from email.message import EmailMessage

# Open the plain text file whose name is in textfile for reading.
with open(textfile) as fp:
    # Create a text/plain message
    msg = EmailMessage()
    msg.set_content(fp.read())

me == sannahziama@gmail.com
you == spziama@gmail.com
msg['Subject'] = 'The contents of %s' % textfile
msg['From'] = me
msg['To'] = you

# Send the message via our own SMTP server.
s = smtplib.SMTP('localhost')
s.send_message(msg)
s.quit()

NameError: name 'textfile' is not defined

In [92]:
#!/usr/bin/env python
# encoding: utf-8
"""
Mailer.py
Created by Robert Dempsey on 11/07/14.
Copyright (c) 2014 Robert Dempsey. All rights reserved.
"""

import sys
import os
import smtplib
from email import encoders
from email.mime.base import MIMEBase
from email.mime.multipart import MIMEMultipart
from email.mime.text import MIMEText

COMMASPACE = ', '

class Mailer:
    def __init__(self, **kwargs):
        self.properties = kwargs

    # Subject
    @property
    def subject(self):
        return self.properties.get('subject', 'None')

    @subject.setter
    def subject(self, s):
        self.properties['subject'] = s

    @subject.deleter
    def subject(self):
        del self.properties['subject']

    # Recipients
    @property
    def recipients(self):
        return self.properties.get('recipients', 'None')

    @recipients.setter
    def recipients(self, r):
        self.properties['recipients'] = r

    @recipients.deleter
    def recipients(self):
        del self.properties['recipients']

    # Send From
    @property
    def send_from(self):
        return self.properties.get('send_from', 'None')

    @send_from.setter
    def send_from(self, s_from):
        self.properties['send_from'] = s_from

    @send_from.deleter
    def send_from(self):
        del self.properties['send_from']

    # Gmail Password
    @property
    def gmail_password(self):
        return self.properties.get('gmail_password', 'None')

    @gmail_password.setter
    def gmail_password(self, g_pass):
        self.properties['gmail_password'] = g_pass

    @gmail_password.deleter
    def gmail_password(self):
        del self.properties['gmail_password']

    # Message
    @property
    def message(self):
        return self.properties.get('message', 'None')

    @message.setter
    def message(self, m):
        self.properties['message'] = m

    @message.deleter
    def message(self):
        del self.properties['message']

    # Attachments
    @property
    def attachments(self):
        return self.properties.get('attachments', 'None')

    @attachments.setter
    def attachments(self, a):
        self.properties['attachments'] = a

    @attachments.deleter
    def attachments(self):
        del self.properties['attachments']

    def send_email(self):
        # Create the enclosing (outer) message
        outer = MIMEMultipart('alternative')
        outer['Subject'] = self.subject
        outer['To'] = COMMASPACE.join(self.recipients)
        outer['From'] = self.send_from

        msg = MIMEBase('application', "octet-stream")

        # Add the text of the email
        email_body = MIMEText(self.message, 'plain')
        outer.attach(email_body)

        # Add the attachments
        if self.attachments != 'None':
            for file in self.attachments:
                try:
                    with open(file, 'rb') as fp:
                        msg.set_payload(fp.read())
                    encoders.encode_base64(msg)
                    msg.add_header('Content-Disposition', 'attachment', filename=os.path.basename(file))
                    outer.attach(msg)
                except:
                    print("Unable to open one of the attachments. Error: ", sys.exc_info()[0])
                    raise

        composed = outer.as_string()

        try:
            with smtplib.SMTP('smtp.gmail.com', 587) as s:
                s.ehlo()
                s.starttls()
                s.ehlo()
                s.login(self.send_from, self.gmail_password)
                s.sendmail(self.send_from, self.recipients, composed)
                s.close()
            print("Email sent!")
        except:
            print("Unable to send the email. Error: ", sys.exc_info()[0])
            raise

def main():
    pass

if __name__ == '__main__': main()

In [105]:
msg = Mailer()

In [106]:
subjt = 'Test'
messg = 'Hi, this is a test'
recipients = "sannahziama@gmail.com"
frm = "sannahziama@gmail.com"
pword = ""


In [107]:
msg.subject = subjt
msg.message = messg
msg.send_from = frm
msg.recipients = recipients

In [110]:
msg.send_email()

Unable to send the email. Error:  <class 'smtplib.SMTPAuthenticationError'>


SMTPAuthenticationError: (535, b'5.7.8 Username and Password not accepted. Learn more at\n5.7.8  https://support.google.com/mail/?p=BadCredentials t127sm5521318pfd.118 - gsmtp')